In [1]:
import pandas as pd

## Task 1 - NRI Data Cleaning

In [23]:
# __1. Import the NRI data. Ensure that the [FIPS code]
# (https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code) 
# variable ('STCOFIPS') is correctly identified as a string / character variable. 
# Otherwise, the leading zeros will be removed.__

# Change the columns type while import the data to make sure the leading 0 are correctly included
NRI = pd.read_csv('NRI_Table_Counties.csv', dtype={'STCOFIPS': str})
NRI['STCOFIPS'].unique()

array(['01001', '01003', '01005', ..., '78010', '78020', '78030'],
      dtype=object)

In [30]:
# __2. Subset the NRI data to include only the 5-digit state/county 
# FIPS code and all colums ending with '\_AFREQ' and '\_RISKR'. 
# Each of these columns represents a different hazard type.
NRI_Sub = NRI.filter(regex='(_AFREQ|_RISKR)$')
NRI_Sub = NRI_Sub.join(NRI[['STCOFIPS']]) ## Also include the 5-digit state/county FIPS Code 

# Make sure the unique value is correct
NRI_Sub['STCOFIPS'].nunique()

3231

In [57]:
# __3. Create a table / dataframe that, for each hazard type, 
# shows the number of missing values in the '\_AFREQ' and '\_RISKR' columns.
NRI_Missing = NRI_Sub.melt(var_name='Column', value_name='Value')
NRI_Missing = NRI_Missing.groupby('Column')['Value'].apply(lambda x: x.isnull().sum()).reset_index()
NRI_Missing['Type'] = NRI_Missing['Column'].str[-5:]
NRI_Missing['Hazard_Type'] = NRI_Missing['Column'].str[:4] 
NRI_Missing = NRI_Missing[NRI_Missing['Column'] != 'STCOFIPS']
NRI_Missing = NRI_Missing.drop(columns={'Column'})

NRI_Missing = pd.pivot_table(NRI_Missing, values=['Value'],
                             index=['Hazard_Type'],
                             columns=['Type'],
                             aggfunc="sum",
                             fill_value=0).reset_index()

NRI_Missing.columns = NRI_Missing.columns.droplevel(0)
new_column_names = ['Hazard_Type', 'Missing_AFREQ', 'Missing_RISKR']
NRI_Missing.columns = new_column_names
print(NRI_Missing)

In [59]:
# __4. Show the cross-tabulation of the 'AVLN_AFREQ' and 'AVLN_RISKR' columns 
# (including missing values). What do you observe?_

cross_tab = pd.crosstab(NRI_Sub['AVLN_AFREQ'], NRI_Sub['AVLN_RISKR'], dropna=False)
NRI_Sub['AVLN_RISKR'].unique()
cross_tab
## Findings: As the AVLN_AFREQ frequency increases, the relatively risk also increases.

AVLN_RISKR,Not Applicable,Relatively High,Relatively Low,Relatively Moderate,Very High,Very Low
AVLN_AFREQ,,,,,,
0.010000,0,0,0,0,0,24
0.016667,0,0,0,0,0,55
0.033333,0,0,14,0,0,15
0.050000,0,0,10,2,0,3
0.066667,0,0,12,4,0,0
0.083333,0,0,5,3,0,0
0.100000,0,0,1,2,0,0
0.116667,0,0,2,4,0,0
0.133333,0,0,1,6,0,0


In [67]:
# __5. Assuming that a risk that is "not applicable" to a county has an annualized frequency of 0, 
# impute the relevant missing values in the '\_AFREQ' columns with 0.
AFREQ_col = NRI_Sub.filter(regex='_AFREQ$')
NRI_Sub[AFREQ_col.columns] = AFREQ_col.fillna(0)
NRI_Sub

## Task 2 - SVI Data Cleaning

In [73]:
# __1. Import the SVI data. Ensure that the FIPS code is correctly identified as a string / character variable.
#  Otherwise, the leading zeros will be removed.__

SVI = pd.read_csv('SVI_2022_US_county.csv', dtype={'FIPS':str})

# __1. Subset the SVI data to include only the following columns:__
# `ST, STATE, ST_ABBR, STCNTY, COUNTY, FIPS, LOCATION, AREA_SQMI, E_TOTPOP, EP_POV150, EP_UNEMP, EP_HBURD, EP_NOHSDP, 
# EP_UNINSUR, EP_AGE65, EP_AGE17, EP_DISABL, EP_SNGPNT, EP_LIMENG, EP_MINRTY, EP_MUNIT, EP_MOBILE, EP_CROWD, EP_NOVEH, 
# EP_GROUPQ, EP_NOINT, EP_AFAM, EP_HISP, EP_ASIAN, EP_AIAN, EP_NHPI, EP_TWOMORE, EP_OTHERRACE`

columns_to_use = [
    'ST', 'STATE', 'ST_ABBR', 'STCNTY', 'COUNTY', 'FIPS', 'LOCATION', 'AREA_SQMI', 'E_TOTPOP', 'EP_POV150', 
    'EP_UNEMP', 'EP_HBURD', 'EP_NOHSDP', 'EP_UNINSUR', 'EP_AGE65', 'EP_AGE17', 'EP_DISABL', 'EP_SNGPNT', 
    'EP_LIMENG', 'EP_MINRTY', 'EP_MUNIT', 'EP_MOBILE', 'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ', 'EP_NOINT', 
    'EP_AFAM', 'EP_HISP', 'EP_ASIAN', 'EP_AIAN', 'EP_NHPI', 'EP_TWOMORE', 'EP_OTHERRACE'
]
SVI_Sub = SVI[columns_to_use]
print(SVI_Sub.columns)

Index(['ST', 'STATE', 'ST_ABBR', 'STCNTY', 'COUNTY', 'FIPS', 'LOCATION',
       'AREA_SQMI', 'E_TOTPOP', 'EP_POV150', 'EP_UNEMP', 'EP_HBURD',
       'EP_NOHSDP', 'EP_UNINSUR', 'EP_AGE65', 'EP_AGE17', 'EP_DISABL',
       'EP_SNGPNT', 'EP_LIMENG', 'EP_MINRTY', 'EP_MUNIT', 'EP_MOBILE',
       'EP_CROWD', 'EP_NOVEH', 'EP_GROUPQ', 'EP_NOINT', 'EP_AFAM', 'EP_HISP',
       'EP_ASIAN', 'EP_AIAN', 'EP_NHPI', 'EP_TWOMORE', 'EP_OTHERRACE'],
      dtype='object')


In [82]:
# __2. Create a table / dataframe that shows the number of missing values in each column.
# (Hint: if you wrote a function for Task 1, you can reuse it here.)

# Function to calculate the number of missing values in each column
def missing_val(table):
    missing = table.isnull().sum()
    return pd.DataFrame(missing, columns=['Missing_Val'])

# Applying the function to the loaded DataFrame
SVI_Missing = missing_val(SVI_Sub)

# Display the resulting DataFrame (Apparently there are no missing values)
SVI_Missing

# Check the original subset table to see if the function was written correctly
SVI_Sub.columns[SVI_Sub.isnull().any()]

Index([], dtype='object')

## Task 3 - Data Merging

In [113]:
# __1. Identify any FIPS codes that are present in the NRI data but not in the SVI data and vice versa. /
# Describe any discrepancies and possible causes? What to these discrepancies, if any, 
# mean for interpreting results based on the merged dataset moving forward?

# First get the FIPS code from both dataset
NRI_FIPS = NRI['STCOFIPS']
SVI_FIPS = SVI['FIPS']

# FIPS codes that are in NRI but not in SVI
# FIPS codes that are in SVI but not in NRI
FIPS_NRI_NotSVI = NRI_FIPS[~NRI_FIPS.isin(SVI_FIPS)]
FIPS_SVI_NotNRI = SVI_FIPS[~SVI_FIPS.isin(NRI_FIPS)]

# See what are the missing gepgraphy in the SVI
FIPS_NRI_NotSVI_check = FIPS_NRI_NotSVI.to_list()
NRI[NRI['STCOFIPS'].isin(FIPS_NRI_NotSVI_check)]['STATE'].unique()

#############################
## Answer: Looks like one state Connecticut and other special terrotories like American Samoa, Guam ... are not included in the SVI dataset. From the website of NRI
## I can see the the NRI dataset includes American Samoa, Commonwealth of the Northern Mariana Islands, Guam, Puerto Rico, and the U.S. Virgin Islands.The missing county in connecticut includes
## all the normal county geo names we see in other dataset. However, the connecticut geo in the SVI dataset are specified as planning region (https://storymaps.arcgis.com/stories/23bc7986213547a79cb8a5dafa84d68d)
## It looks like the plannign regions in CT are being treated by tehe offical County equivalents by the Census Bureau due to the file changes by the CT state. 

# See what are the missing gepgraphy in the NRI
FIPS_SVI_NotNRI_check = FIPS_SVI_NotNRI.to_list()
SVI[SVI['FIPS'].isin(FIPS_SVI_NotNRI_check)]

#############################
## Answer: looks like NRI does not have special County geo like planning region that shows in the table in the connecticut. As stated above, SVI uses planning regiong for the County equivalent
## as apposed to the NRI uses of traditional county definition.

#############################
## Answer: These results are different might be due to the fact that data collected by the SVI better reflect the census bureau geo definition and the year of data collected. Howeverm for the state of connecticut,
## we might need to do some geoanalysis to map the traditional county to the planning region in order to generalize the analysis. Otherwise, the CT analysis will create discrepency since the planning region
## seems better reflect the devlopment and demographics of the population in the state, but not the traditional county definition.

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
309,9,Connecticut,CT,9110,Capitol Planning Region,09110,"Capitol Planning Region, Connecticut",1027.329337,977165,0,...,5.8,0.1,0.1,0.1,0.0,0.1,3.3,0.2,0.5,0.1
310,9,Connecticut,CT,9120,Greater Bridgeport Planning Region,09120,"Greater Bridgeport Planning Region, Connecticut",140.181229,326381,0,...,5.4,0.2,0.1,0.1,0.0,0.1,2.7,0.4,1.2,0.3
311,9,Connecticut,CT,9130,Lower Connecticut River Valley Planning Region,09130,Lower Connecticut River Valley Planning Region...,424.114635,175244,0,...,3.4,0.2,0.1,0.1,0.0,0.1,3.6,0.5,0.2,0.1
312,9,Connecticut,CT,9140,Naugatuck Valley Planning Region,09140,"Naugatuck Valley Planning Region, Connecticut",412.784010,451887,0,...,4.0,0.2,0.0,0.1,0.0,0.1,4.3,0.4,0.6,0.2
313,9,Connecticut,CT,9150,Northeastern Connecticut Planning Region,09150,"Northeastern Connecticut Planning Region, Conn...",553.891686,95687,0,...,1.4,0.2,0.1,0.1,0.1,0.1,3.6,0.4,0.2,0.1
314,9,Connecticut,CT,9160,Northwest Hills Planning Region,09160,"Northwest Hills Planning Region, Connecticut",786.641459,112696,0,...,2.5,0.2,0.0,0.1,0.0,0.1,3.8,0.7,0.4,0.2
315,9,Connecticut,CT,9170,South Central Connecticut Planning Region,09170,"South Central Connecticut Planning Region, Con...",367.168772,571298,0,...,4.2,0.1,0.1,0.1,0.0,0.1,2.6,0.2,0.6,0.2
316,9,Connecticut,CT,9180,Southeastern Connecticut Planning Region,09180,"Southeastern Connecticut Planning Region, Conn...",598.134929,280293,0,...,3.8,0.2,0.4,0.1,0.0,0.1,4.2,0.4,0.5,0.2
317,9,Connecticut,CT,9190,Western Connecticut Planning Region,09190,"Western Connecticut Planning Region, Connecticut",532.116655,620666,0,...,5.2,0.1,0.1,0.1,0.0,0.1,3.6,0.3,0.8,0.2


In [122]:
# __2. Merge the NRI and SVI data on the FIPS code. Use an outer join to keep all counties in the final dataset.
Merge_SVI_NRI = pd.merge(NRI, SVI, left_on='STCOFIPS', right_on='FIPS', how= 'outer')

,OID_,NRI_ID,STATE_x,STATEABBRV,STATEFIPS,COUNTY_x,COUNTYTYPE,COUNTYFIPS,STCOFIPS,POPULATION,...,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
0,1.0,C01001,Alabama,AL,1.0,Autauga,County,1.0,01001,58764.0,...,1.1,0.4,0.1,0.1,0.0,0.1,3.3,1.0,0.2,0.3
1,2.0,C01003,Alabama,AL,1.0,Baldwin,County,3.0,01003,231365.0,...,0.9,0.1,0.2,0.1,0.0,0.1,3.1,0.4,0.4,0.3
2,3.0,C01005,Alabama,AL,1.0,Barbour,County,5.0,01005,25160.0,...,0.5,0.1,0.3,0.1,0.0,0.1,1.8,0.7,1.2,0.8
3,4.0,C01007,Alabama,AL,1.0,Bibb,County,7.0,01007,22239.0,...,0.3,0.4,0.1,0.1,0.0,0.2,1.7,1.0,0.1,0.1
4,5.0,C01009,Alabama,AL,1.0,Blount,County,9.0,01009,58992.0,...,0.2,0.2,0.1,0.1,0.2,0.2,2.8,0.7,0.1,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3235,3227.0,C72151,Puerto Rico,PR,72.0,Yabucoa,Municipio,151.0,72151,30397.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3236,3228.0,C72153,Puerto Rico,PR,72.0,Yauco,Municipio,153.0,72153,34151.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3237,3229.0,C78010,Virgin Islands,VI,78.0,St. Croix,Island,10.0,78010,40913.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3238,3230.0,C78020,Virgin Islands,VI,78.0,St. John,Island,20.0,78020,3882.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [124]:
# __3. Create a table / dataframe that shows the number of missing values in each column of the merged dataset.
# Applying the previous created function
Merge_SVI_NRI_missing = missing_val(Merge_SVI_NRI)
Merge_SVI_NRI_missing

,Missing_Val
OID_,9
NRI_ID,9
STATE_x,9
STATEABBRV,9
STATEFIPS,9
...,...
MP_NHPI,96
EP_TWOMORE,96
MP_TWOMORE,96
EP_OTHERRACE,96


## Task 4 - Data Analysis

In [ ]:
# __1. For each numerical variable in the merged dataset, plot a histogram showing the distribution of values.
# (Hint: write a function to make the histogram for a single variable, then use a loop or apply function to make the histograms for all numerical variables.)